# MNIST Classifier

<center>
    <img alt="MNIST Handwritten Digits Image" src="https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png">
</center>

In this example, we will predict handwritten digits in the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) using a [multi-layer perceptron (MLP)](https://en.wikipedia.org/wiki/Multilayer_perceptron). This is similar to the tutorial notebook, but with an added comparison with a standard model.

Of course, other neural network architectures such as [convolutional neural networks (CNNs)](https://en.wikipedia.org/wiki/Convolutional_neural_network) are better suited for this task, but for this example we will stick with MLPs.

## Setup

First, let's prepare the imports and set the keras backend.

In [1]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [2]:
import keras
import numpy as np

Define constants relating to the data.

In [3]:
NUM_CLASSES = 10        # 10 distinct classes, 0 to 9
INPUT_SHAPE = (28, 28)  # 28 x 28 greyscale images

Load the data from the `mnist` dataset.

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

Perform some preprocessing.

In [5]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

## Defining the Model

As mentioned, we will be using a MLP for the model. However, instead of using `keras`'s default `Dense` layer, we will use `keras_mml`'s `DenseMML` layer (which stands for Dense Matrix-Multiplication-less). `DenseMML` is designed to be a direct replacement for `Dense` layers in fully-connected layers, so we don't have to change the architecture of the model much.

In [6]:
import keras_mml

Define the `Sequential` model.

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),  # The last layer needs to be `Dense` for the output to work
    ],
    name="Classifier-MML"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
model.summary()

Model: "Classifier-MML"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml (DenseMML)            │ (None, 256)            │       200,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml_1 (DenseMML)          │ (None, 256)            │        65,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_mml_2 (DenseMML)          │ (None, 256)            │        65,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 334,346 (1.28 MB)

 Trainable params: 334,346 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We can now train the model.

In [9]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6645 - loss: 1.5924 - val_accuracy: 0.8913 - val_loss: 0.4371
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8849 - loss: 0.4253 - val_accuracy: 0.9162 - val_loss: 0.3054
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9046 - loss: 0.3355 - val_accuracy: 0.9257 - val_loss: 0.2674
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9109 - loss: 0.3105 - val_accuracy: 0.9275 - val_loss: 0.2519
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9144 - loss: 0.2935 - val_accuracy: 0.9322 - val_loss: 0.2397
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9197 - loss: 0.2803 - val_accuracy: 0.9290 - val_loss: 0.2413
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9193 - loss: 0.2796 - val_accuracy: 0.9325 - val_loss: 0.2390
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9207 - loss: 0.2673 - val_accuracy: 0.

Once the model is trained, let's evaluate it.

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.2564573884010315
Test accuracy: 0.9257000088691711


Not bad!

## A Comparison - An MLP Using Normal `Dense` Layers

Let's compare our model's performance to a model that uses the regular `Dense` layers.

In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),
    ],
    name="Classifier-Normal"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.summary()

Model: "Classifier-Normal"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,114 (1.28 MB)

 Trainable params: 335,114 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We'll train the model using the same `batch_size`, `epochs`, and `validation_split`.

In [13]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8434 - loss: 0.5223 - val_accuracy: 0.9215 - val_loss: 0.2734
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9046 - loss: 0.3310 - val_accuracy: 0.9263 - val_loss: 0.2530
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9099 - loss: 0.3198 - val_accuracy: 0.9237 - val_loss: 0.2767
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9143 - loss: 0.3064 - val_accuracy: 0.9267 - val_loss: 0.2601
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9125 - loss: 0.3049 - val_accuracy: 0.9240 - val_loss: 0.2621
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9162 - loss: 0.2900 - val_accuracy: 0.9258 - val_loss: 0.2653
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9182 - loss: 0.2914 - val_accuracy: 0.9243 - val_loss: 0.2656
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9210 - loss: 0.2814 - val_accuracy: 0

Again, we evaluate the model.

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.30377599596977234
Test accuracy: 0.9165999889373779


Notice that the accuracy of the normal model is quite close to the accuracy of the MML model. With a slight decrease in accuracy, the model itself does not use matrix multiplications at all, and also reduces memory usage.